# Customer Segmentation by Dongmei Yin

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

def customerSegmentation(filepath):
    # Part1: read in raw data with provided filepah
    aisle = pd.read_csv(filepath + '/aisles.csv')
    department = pd.read_csv(filepath + '/departments.csv')
    order_products = pd.read_csv(filepath + '/sample_order_products.csv')
    orders = pd.read_csv(filepath + '/sample_order.csv')
    products = pd.read_csv(filepath + '/products.csv')
    aisle_match = pd.read_csv(filepath + '/final_aisle_match.csv');
    
    # Part 2: data preprocessing
    
    # Take out orderID and productID from both train and prior table and merge them into one order_products_id table
    order_products_id = order_products[['order_id','product_id']]
    # take out orderID and userID from orders table
    order_user_id = orders[['order_id','user_id']]
    # get total order count for each user
    user_order_count = order_user_id[['user_id','order_id']]
    user_order_count = user_order_count.groupby(['user_id']).agg({'order_id': 'count'}).reset_index()
    user_order_count.rename(columns={"order_id": "order_count"}, inplace=True)
    #combine order_products and products table to see what aisle each product belongs for each order
    order_product_aisle = order_products_id.merge(products,on='product_id')
    #from order_product_aisle table, only select orderID and aisleID to aggregate some data since we only deal with 
    #aisle level granularity
    #In addition, count of products from each aisle for each order is kept
    order_aisle_id = order_product_aisle[['order_id','aisle_id']]
    order_aisle_id['aisle_id2'] = order_product_aisle['aisle_id']
    order_aisle_count = order_aisle_id.groupby(['order_id','aisle_id']).agg({'aisle_id2': 'count'}).reset_index()
    order_aisle_count.rename(columns={"aisle_id2": "product_count"}, inplace=True)
    #join order_user_id table and order_aisle_count table to see which order belongs to which user
    order_user_aisle = order_aisle_count.merge(order_user_id, on='order_id')
    #from order_user_aisle table, take out orderID and group by user_id, aisle_id
    user_aisle_id = order_user_aisle[['user_id','aisle_id','product_count']]
    user_aisle_count = user_aisle_id.groupby(['user_id','aisle_id']).agg({'product_count': 'sum'}).reset_index()
    # user_aisle_count right join user_order_count 
    user_aisle_order_count= user_aisle_count.merge(user_order_count, how='right',on='user_id')
    # final aisle grouping along with filtration of essential aisles
    new_user_aisle_order_count = user_aisle_order_count.merge(aisle_match, how='inner', on='aisle_id')
    new_user_aisle_order_count = new_user_aisle_order_count[['user_id','new_aisle_ID','product_count','order_count']]
    new_user_aisle_order_count = new_user_aisle_order_count.groupby(['user_id','new_aisle_ID']).agg({'product_count':'sum','order_count':'sum'}).reset_index()
    # divide product count by order count to get for aisle, how many products on average each customer would buy per order
    new_user_aisle_order_count['avg'] = new_user_aisle_order_count['product_count']/new_user_aisle_order_count['order_count']
    user_aisle_avgCount_annual = new_user_aisle_order_count[['user_id','new_aisle_ID','avg']]
    # get final data
    data = pd.pivot_table(user_aisle_avgCount_annual, values='avg', index=['user_id'],columns=['new_aisle_ID'], aggfunc=np.sum, fill_value=0)
    data.reset_index(inplace=True)
                                                                                                    
    # index match
    index_match = data['user_id']
    data.set_index('user_id', inplace=True)
    
    # Model prediction
    # apply principal component analysis to reduce the dimension from 69 to 34 dimensions
    pca = PCA(n_components=34)
    pca_34 = pca.fit_transform(data)
    pca_34_90percent = pd.DataFrame(pca_34)
                                                                                                     
    # load the model from disk and predict on the data
    filename = 'customer_segmentation_model.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    y_kmean = loaded_model.predict(pca_34_90percent)
    result = pca_34_90percent.copy(deep=True)
    result.reset_index(inplace=True)
    result.insert(1,'label',y_kmean)
    result = result.rename(columns = {'index':'X'})
    result = result[['X','label']]
    index_match = pd.DataFrame(index_match)
    index_match.reset_index(inplace=True)
    index_match = index_match.rename(columns = {'index':'X'})
    result = index_match.merge(result, on = 'X')
    result = result[['user_id','label']]
                                                                                                     
    return result



# basket analysis (Apriori version) by Yilin Zhou

In [2]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

def basket_analysis(filepath):
    aisle = pd.read_csv(filepath + '/aisles.csv')
    department = pd.read_csv(filepath + '/departments.csv')
    order_products = pd.read_csv(filepath + '/sample_order_products.csv')
    orders = pd.read_csv(filepath + '/sample_order.csv')
    products = pd.read_csv(filepath + '/products.csv')
    
#     def transform_frequentItem_to_product(frequent_itemsets):
#         arr_relation=frequent_itemsets.values
#         final_arr = []
#         for item in arr_relation:
#             arr = []
#             for it in item[1]:
#                 arr.append(products[products['product_id' ]== it]['product_name'].values[0])
#             final_arr.append(arr)
#         return final_arr
    
    # order,product and user table
    order_user_id = orders[['order_id','user_id']]
    user_product_id=order_user_id.merge(order_products,left_on='order_id', right_on='order_id')
    # find top 3 aisle_id
    aisle_ids = []
    aisle_arr = aisle[aisle['aisle'].isin(['fresh fruits','fresh vegetables','packaged vegetables fruits'])].values
    for item in aisle_arr:
        aisle_ids.append(item[0])
    # product in those 3 aisle
    product_ids = []
    product_arr = products[products['aisle_id'].isin(aisle_ids)].values
    for item in product_arr:
        product_ids.append(item[0])
    # user,product and user table after removing those products
    user_product_id = user_product_id[~user_product_id['product_id'].isin(product_ids)]
    return user_product_id
    
def returnProduct(filepath,i,minSup,minCon):
    aisle = pd.read_csv(filepath + '/aisles.csv')
    department = pd.read_csv(filepath + '/departments.csv')
    order_products = pd.read_csv(filepath + '/sample_order_products.csv')
    orders = pd.read_csv(filepath + '/sample_order.csv')
    products = pd.read_csv(filepath + '/products.csv')
    def Apriori(user_product_part1):
    
        user_product_dic={}
        for item in user_product_part1.itertuples():
            if(user_product_dic.get(item[2])):
                user_product_dic[item[2]].append(item[3])
            else:
                user_product_dic[item[2]] = [] 
                user_product_dic[item[2]].append(item[3])
        transaction = []
        for item in user_product_dic:
            transaction.append(user_product_dic[item])
    #     print('transaction',transaction)
        te = TransactionEncoder()
        te_ary = te.fit(transaction).transform(transaction)
    #     print('te_ary',te_ary)
        df = pd.DataFrame(te_ary, columns=te.columns_)
        return df
    
    def transform_associationRule_to_product(association_rule):
        relationship_rule = association_rule[['antecedents','consequents']]
        arr_relation=relationship_rule.values
        final_rules = []
        for item in arr_relation:
            pre_product_Arr = []
            after_product_Arr = []
            pre = item[0]
            after = item[1]
            for pre_item in pre:
                pre_product_Arr.append(products[products['product_id' ]== pre_item]['product_name'].values[0])
            for after_item in after:
                after_product_Arr.append(products[products['product_id' ]== after_item]['product_name'].values[0])
            tup = (pre_product_Arr,'----->',after_product_Arr)
            final_rules.append(tup)
        return final_rules
    
    customercluster = customerSegmentation(fp);
    user_product_id = basket_analysis(fp);
    ids = customercluster[customercluster['label']==i]['user_id'].values
    user_product_part0 = user_product_id[user_product_id['user_id'].isin(ids)]
    df = Apriori(user_product_part0)
    frequent_itemsets0 = apriori(df, min_support=minSup, use_colnames=True)
#     print('frequent_itemsets0',frequent_itemsets0)
    association_rule0 = association_rules(frequent_itemsets0, metric="confidence", min_threshold=minCon)
#     print('association_rule0',association_rule0)
    association_rule_trans = transform_associationRule_to_product(association_rule0)
    for item in association_rule_trans:
        print(item)
#     print(association_rule_trans)

In [18]:
# fp = './Sample Data'
# returnProduct(fp,0,0.25,0.25)

In [19]:
# returnProduct(fp,1,0.08,0.25)

In [20]:
# returnProduct(fp,2,0.4,0.4)

In [21]:
# returnProduct(fp,3,0.3,0.3)

In [22]:
# returnProduct(fp,4,0.5,0.5)

# Code to run whole system

In [8]:
# file path for the sample data folder
# fp = 'C:/Users/lyn02/Desktop/CMPE255 Project/Sample Data';
fp = './Sample Data'
# getting customer clusters with two columns (user_id, label)
customercluster = customerSegmentation(fp);

/Users/zhouyilin/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
customercluster
# user_id , product 

,user_id,label
0,21,1
1,28,1
2,36,1
3,54,1
4,63,1
...,...,...
245,203750,1
246,204090,1
247,204236,1
248,205273,1


In [16]:
def returnRecommandItem(filepath):
    apriori_output = pd.read_csv(filepath + '/output_Aprioir.csv');
    apriori_output = apriori_output[['first','second','label']]
    
    aisle = pd.read_csv(filepath + '/aisles.csv')
    department = pd.read_csv(filepath + '/departments.csv')
    order_products = pd.read_csv(filepath + '/sample_order_products.csv')
    orders = pd.read_csv(filepath + '/sample_order.csv')
    products = pd.read_csv(filepath + '/products.csv')
    aisle_match = pd.read_csv(filepath + '/final_aisle_match.csv');
    
    order_user_id = orders[['order_id','user_id']]
    user_product_id=order_user_id.merge(order_products,left_on='order_id', right_on='order_id')
    user_product_id = user_product_id[['user_id','product_id']]
    user_product_label = customercluster.merge(user_product_id,left_on='user_id', right_on='user_id')
#     user_product_label_arr = user_product_label.values

    user_product_dic={}
    for item in user_product_label.itertuples():
#         print(item)
        if(user_product_dic.get(item[1])):
            user_product_dic[item[1]].append(item[3])
        else:
            user_product_dic[item[1]] = []
            user_product_dic[item[1]].append(item[3])
#     transaction = []
#     for item in user_product_dic:
#         transaction.append(user_product_dic[item])
    predict_dic = {}
    for item in user_product_dic:
        predict_set=[]
        pids = user_product_dic[item]
        label = user_product_label[user_product_label['user_id']==item]['label'].values[0]
        apriori_output_part = apriori_output[apriori_output['label']==label]
#         print(pids)
        for item2 in apriori_output_part.itertuples():
#             print(item2[1].strip('[]'))
            if(len(item2[1].strip('[]'))<7):
                if(int(item2[1].strip('[]')) in pids):
                    if(len(item2[2].strip('[]'))<7):
                        predict_set.append(int(item2[2].strip('[]')))
                    
        predict_dic[item] = predict_set
    for item in predict_dic:
        pro_arr = []
        for item2 in predict_dic[item]:
            pro_arr.append(products[products['product_id' ]== item2]['product_name'].values[0])
        predict_dic[item] = pro_arr
    return predict_dic

In [17]:
fp = './Sample Data'
# getting customer clusters with two columns (user_id, label)
predict_dic = returnRecommandItem(fp)
predict_dic

{21: ['Organic Whole Milk'],
 28: ['Organic Whole String Cheese',
  'Organic Whole String Cheese',
  'Organic Whole Milk',
  'Organic Whole Milk',
  'Organic Whole Milk',
  'Organic Cilantro',
  'Grated Parmesan',
  'Organic Italian Parsley Bunch',
  'Spinach Peas & Pear Stage 2 Baby Food',
  'Baby Food Stage 2 Blueberry Pear & Purple Carrot',
  'Organic Italian Parsley Bunch',
  'Spinach Peas & Pear Stage 2 Baby Food',
  'Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
 36: [],
 54: ['Organic Whole String Cheese',
  'Organic Whole String Cheese',
  'Organic Whole String Cheese',
  'Organic Whole Milk',
  'Organic Whole Milk',
  'Organic Whole Milk',
  'Organic Whole Milk',
  'Organic Whole Milk',
  'Organic Italian Parsley Bunch',
  'Extra Virgin Olive Oil',
  'Spinach Peas & Pear Stage 2 Baby Food',
  'Baby Food Stage 2 Blueberry Pear & Purple Carrot',
  'Organic Cilantro',
  'Organic Cilantro',
  'Organic Italian Parsley Bunch',
  'Grated Parmesan',
  'Spinach Peas & Pear Stage 